In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from time import sleep

In [2]:
'''
We need information on each newsletter from each category 
(can be found by clicking on a category and then clicking all). 
Information we need scraped are name of newsletter, link to newsletter, and name of author.
'''

'\nWe need information on each newsletter from each category \n(can be found by clicking on a category and then clicking all). \nInformation we need scraped are name of newsletter, link to newsletter, and name of author.\n'

In [44]:
def get_info(an_article):
    newsletter_dict = {}
    newsletter_dict["newsletter_name"] = an_article.find("div", {"class": "publication-title"}).text.strip()
    newsletter_dict["url"]             = an_article["href"]
    author                             = an_article.find("div", {"class": "publication-author"}).text
    if "by" not in author:
        author = ""
    else:
        author = author.split("·")
        author = author[0]
        author = author.replace("by", "", 1)
        author = author.strip()

    newsletter_dict["author"] = author
    return newsletter_dict


In [38]:
def get_buttons(driver):
    html           = driver.page_source
    soup           = BeautifulSoup(html, "html.parser")
    categories     = soup.find("div", {"class": "categories"})
    category_names = categories.find_all("button", {"class": "button"})
    buttons        = []

    for name in category_names:
        buttons.append(name.text)
    clickables = {}
    for btn in buttons:
        element = driver.find_element_by_xpath(f"//*[contains(text(), '{btn}')]")
        clickables[btn] = element

    del clickables["Featured"]
    return clickables

In [39]:
def scroll():
    try:
        view_btn = driver.find_element_by_xpath(f"//*[contains(text(), 'View more')]")
        
        while view_btn:
            view_btn.click()
            sleep(5)
            view_btn = driver.find_element_by_xpath(f"//*[contains(text(), 'View more')]")
    except:
        print("done scrolling for",end=" ")
    

In [6]:
# type(driver.find_element_by_xpath(f"//*[contains(text(), 'View')]"))

In [40]:
url    = "https://substack.com/"
driver = webdriver.Chrome("../../chromedriver")
driver.get(url)

buttons = get_buttons(driver)

In [42]:
master_list = []

for title, btn in buttons.items():
    btn.click()
    sleep(1)
    all_btn = driver.find_element_by_xpath(f"//*[contains(text(), 'All')]")
    all_btn.click()
    sleep(2.5)
    scroll()
    print(title, end=" ")
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    publications = soup.find("div", {"class": "publications"})
    articles = publications.find_all("a", {"class": "publication"})
    print(len(articles))
    for an_article in articles:
        tmp = get_info(an_article)
        tmp["category"] = title
        master_list.append(tmp)

done scrolling for Culture 965
done scrolling for Politics 494
done scrolling for Technology 683
done scrolling for Climate 104
done scrolling for Business 1514
done scrolling for Food & Drink 190
done scrolling for Sports 228
done scrolling for Faith 36
done scrolling for News 92
done scrolling for Literary 526
done scrolling for Art & Illustration 131
done scrolling for Health 175


In [35]:
newsletter_df = pd.DataFrame(master_list)

In [36]:
newsletter_df.to_csv("./sample_output.csv", index=False)

In [43]:
len(master_list)

5138

In [47]:
driver.close()

In [49]:
newsletter_df.describe()

,newsletter_name,url,author,category
count,5140,5140,5140,5140
unique,5116,5140,4102,12
top,BuyUcoin’s Newsletter,https://pdfchomper.substack.com?utm_source=dis...,,Business
freq,4,1,929,1520


In [15]:
newsletter_df

,newsletter_name,url,author,category
0,Culture Study,https://annehelen.substack.com?utm_source=disc...,Anne Helen Petersen,Culture
1,Maybe Baby,https://haleynahman.substack.com?utm_source=di...,Haley Nahman,Culture
2,The Audacity.,https://audacity.substack.com?utm_source=discover,Roxane Gay,Culture
3,Hung Up,https://hunterharris.substack.com?utm_source=d...,Hunter Harris,Culture
4,Webworm with David Farrier,https://www.webworm.co?utm_source=discover,,Culture
...,...,...,...,...
3072,Cosmic Egg,https://cosmicegg.substack.com?utm_source=disc...,Jane Riccobono,Health
3073,Journal of Armchair Medicine,https://amrchairmed.substack.com?utm_source=di...,Abhi Pandey,Health
3074,Everyday Fitness,https://everydayfitness.substack.com?utm_sourc...,Aaron Griffin 🏋️,Health
3075,Venova Wellness,https://paulkeefe.substack.com?utm_source=disc...,Paul Keefe,Health


In [19]:
culture = newsletter_df[newsletter_df.category == "Culture"]
culture

,newsletter_name,url,author,category
0,Culture Study,https://annehelen.substack.com?utm_source=disc...,Anne Helen Petersen,Culture
1,Maybe Baby,https://haleynahman.substack.com?utm_source=di...,Haley Nahman,Culture
2,The Audacity.,https://audacity.substack.com?utm_source=discover,Roxane Gay,Culture
3,Hung Up,https://hunterharris.substack.com?utm_source=d...,Hunter Harris,Culture
4,Webworm with David Farrier,https://www.webworm.co?utm_source=discover,,Culture
5,The Shatner Chatner,https://www.shatnerchatner.com?utm_source=disc...,Daniel Lavery,Culture
6,And It Don't Stop,https://robertchristgau.substack.com?utm_sourc...,Robert Christgau,Culture
7,books/snacks/softcore,https://bitchesgottaeat.substack.com?utm_sourc...,samantha irby,Culture
8,Crème de la Crème,https://aminatou.substack.com?utm_source=discover,Aminatou Sow,Culture
9,"Thinking Better, Together",https://zatrana.substack.com?utm_source=discover,Zat Rana,Culture
